#  Library  

In [16]:
from nrclex import NRCLex

import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GK999CK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# GUI to upload the file and store it in the dataframe

In [17]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import pandas as pd
from pandas.errors import EmptyDataError 
import os

# initalise the tkinter GUI
root = tk.Tk()

root.geometry("500x500")
root.pack_propagate(False) 
root.resizable(0, 0) 

# Frame for TreeView
frame1 = tk.LabelFrame(root, text="Input Data")
frame1.place(height=250, width=500)

# Frame for open file dialog
file_frame = tk.LabelFrame(root, text="Open File")
file_frame.place(height=100, width=400, rely=0.65, relx=0)

# Buttons
button1 = tk.Button(file_frame, text="Load File",command=lambda: Load_excel_data() )
button1.place(rely=0.65, relx=0.50)

button2 = tk.Button(file_frame, text="Browse File", command=lambda: File_dialog())
button2.place(rely=0.65, relx=0.30)

x = tk.Label(root, text = "**Close pop-up after loading file**",fg='red',font=("Arial", 10))
x.place(relx=0.5, rely=0.5,anchor ='center')

# The file/file path text
label_file = ttk.Label(file_frame, text="No File Selected")
label_file.place(rely=0, relx=0)


## Treeview Widget
tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1, relwidth=1) 

treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview)
treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview)
tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) 
treescrollx.pack(side="bottom", fill="x")
treescrolly.pack(side="right", fill="y") 


def File_dialog():
    #This Function will open the file explorer and assign the chosen file path to label_file
    filename = filedialog.askopenfilename(initialdir="/",
                                          title="Select A File",
                                          filetype=(("csv files", "*.csv"),("excel Files", "*.xlsx")))
    label_file["text"] = filename
    return None


def Load_excel_data():
    #If the file selected is valid this will load the file
    file_path = label_file["text"]
    global excel_filename
    try:
        excel_filename = r"{}".format(file_path)
        global data
        if excel_filename[-4:] == ".csv":
            cols_to_use =[0]
            data = pd.read_csv(excel_filename, header=None,usecols = cols_to_use)
        else:
            data = pd.read_excel(excel_filename, header=None, usecols='A') 
        if data.empty==True:
                tk.messagebox.showerror("Information", "File is empty.Please upload file with data")
    except PermissionError:
        tk.messagebox.showerror("Information", "Please close the file and upload again")
        return None
    except EmptyDataError:
        tk.messagebox.showerror("Information", "File is empty.Please upload file with data")
        return None
    except:
        tk.messagebox.showerror("Information", "Please upload the file again.")

    
    clear_data()
    tv1["column"] = list(data.columns)
    tv1["show"] = "headings"

    data_rows = data.to_numpy().tolist()
    for row in data_rows:
        tv1.insert("", "end", values=row) 
    return None


def clear_data():
    tv1.delete(*tv1.get_children())
    return None


root.mainloop()

# Data Prepocessing

In [18]:
data.index = np.arange(1, len(data) + 1)
data.rename(columns={0: 'Input Text'}, inplace=True)
data

,Input Text
1,text
2,This company is for those people who would lik...
3,Your work has been very well appreciated. And ...
4,Your salary will not increase with your experi...
5,Young individuals start their career in Conver...
...,...
4924,- normal day with lot of responsibilities\r - ...
4925,- Nice to working with Infosys \r - Learned ::...
4926,- I learned the corporate life and structure o...
4927,- Great work culture\r - Flexible working cond...


In [19]:
#Cleaning the text
try:
    data['Clean Text'] = data['Input Text'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.replace("'!','#','$','%', '&','@','[',']',' ',']',''",'').str.replace('_',' ').str.strip()
except:
    tk.messagebox.showerror("Information", "Please upload the file again.")

C:\Users\GK999CK\AppData\Local\Temp\ipykernel_31204\32343929.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Clean Text'] = data['Input Text'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.replace("'!','#','$','%', '&','@','[',']',' ',']',''",'').str.replace('_',' ').str.strip()


In [20]:
#converting datatype of clean column to string
try:
    data['Clean Text']=data['Clean Text'].astype(str)
    data
except:
    tk.messagebox.showerror("Information", "Please check the error")

# Detect emotions: dataframe

In [21]:
df = data
df = df[1:]
df = pd.DataFrame(df,columns=['Clean Text'])
df.head()

,Clean Text
2,this company is for those people who would lik...
3,your work has been very well appreciated and y...
4,your salary will not increase with your experi...
5,young individuals start their career in conver...
6,you will never feel like not going to work if ...


# Make sure you process the data: lemmatization etc (not shown here)

In [22]:
df['emotions'] = df['Clean Text'].apply(lambda x: NRCLex(x).affect_frequencies)
df.head()

,Clean Text,emotions
2,this company is for those people who would lik...,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
3,your work has been very well appreciated and y...,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
4,your salary will not increase with your experi...,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."
5,young individuals start their career in conver...,"{'fear': 0.0625, 'anger': 0.0, 'anticip': 0.0,..."
6,you will never feel like not going to work if ...,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't..."


In [23]:
df = pd.concat([df.drop(['emotions'], axis = 1), df['emotions'].apply(pd.Series)], axis = 1)
df.head()

,Clean Text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
2,this company is for those people who would lik...,0.0000,0.0,0.0,0.000000,0.000000,1.000000,0.0000,0.0000,0.0,0.000000,NaN
3,your work has been very well appreciated and y...,0.0000,0.0,0.0,0.272727,0.090909,0.272727,0.0000,0.0000,0.0,0.181818,0.181818
4,your salary will not increase with your experi...,0.0000,0.0,0.0,0.153846,0.076923,0.384615,0.0000,0.0000,0.0,0.153846,0.230769
5,young individuals start their career in conver...,0.0625,0.0,0.0,0.062500,0.062500,0.250000,0.0625,0.0625,0.0,0.125000,0.312500
6,you will never feel like not going to work if ...,0.0000,0.0,0.0,0.250000,0.000000,0.250000,0.0000,0.0000,0.0,0.250000,0.250000


In [24]:
df.to_excel("output.xlsx")

In [40]:
##summary of emotions

fear_list=['anticipation','fear']
df['Fear']=df[fear_list].sum(axis=1)

negative_list=['negative','disgust']
df['Negative']=df[negative_list].sum(axis=1)

joy_list=['trust','joy']
df['Joy']=df[joy_list].sum(axis=1)

positive_list=['surprise','positive']
df['Positive']=df[positive_list].sum(axis=1)

df.rename(columns={'anger':'Anger','sadness':'Sadness'},inplace=True)


df_new=df[['Clean Text','Fear','Joy','Anger','Sadness','Negative','Positive']]
df_new.to_excel("output2.xlsx")